
Multilayer Perceptron Algorithm Feedforward and Backpropagation Passes
======================================================================

This demonstrates an implementation of a neural network using computational graphs and matrix operations
--------------------------------------------------------------------------------------------------------

Author: Joseph Emmanuel Dayo

So that the dilpack package containing the main source code can be reference make sure to do:

```bash
pip install -e .
```

on the project directory first

In [1]:
import numpy as np
from dilpack.learn.computational.nodes import Node, MatMul, Add, Sigmoid, Tanh, LeakyRelu
from dilpack.learn.computational.backpropagate import backpropagate, update
import os
import random

Define the neural network weights, biases

In [2]:
input = Node(np.array([[0.4, 0.5, -0.7]]))

learning_rate = 0.85
biases_layer1 = Node(np.array([[-0.7, -0.8, -0.9, -0.4]]), trainable=True)
weights_layer1 = Node(np.transpose(np.array([[0.1, 0.4, -0.6],
                           [0.3, 0.7, -0.4],
                           [-0.9, 0.3, -0.6],
                           [-0.8, 0.7, 0.2]])), trainable=True)

biases_layer2 = Node(np.array([[-0.7, -0.6, 0.4]]), trainable=True)

weights_layer2 = Node(np.transpose(np.array([[-0.3 ,0.8 ,-0.9 ,0.4],
                           [-0.4 ,-0.2 ,0.7 ,-0.3],
                           [0.4,-0.7,0.8, -0.3]])), trainable=True)

biases_output = Node(np.array([[-0.6,-0.4]]), trainable=True)
weights_output = Node(np.transpose(np.array([[0.8, -0.3, 0.7],
                        [-0.6, 0.7, 0.3]])), trainable=True)


Define the computational graph for the neural network model

In [3]:
output_layer1 = LeakyRelu(Add(MatMul(input, weights_layer1), biases_layer1))
output_layer2 = LeakyRelu(Add(MatMul(output_layer1, weights_layer2), biases_layer2))
output = Sigmoid(Add(MatMul(output_layer2, weights_output), biases_output), logistic_slope=0.7)

Dump the structure of the network:

In [4]:
output.dump()

 SIGMOID(logistic-slope: 0.7)
   input 1:
    Add(A,B)
      input 1:
        MatMul(A,B)
          input 1:
            LeakyRelu(A,slope=0.01)
              input 1:
                Add(A,B)
                  input 1:
                    MatMul(A,B)
                      input 1:
                        LeakyRelu(A,slope=0.01)
                          input 1:
                            Add(A,B)
                              input 1:
                                MatMul(A,B)
                                  input 1:
                                    Value(Trainable=False)
                                      input 1:
                                      [[ 0.4  0.5 -0.7]]
                                  input 2:
                                    Value(Trainable=True)
                                      input 1:
                                      [[ 0.1  0.3 -0.9 -0.8]
 [ 0.4  0.7  0.3  0.7]
 [-0.6 -0.4 -0.6  0.2]]
                              input 2:
             

Perform feedforward pass

In [5]:
predicted_values = output.forward()
print(predicted_values)

[[0.44312378 0.45095094]]


In [6]:
expected_output = np.array([[0.83,0.74]])
error = (expected_output - predicted_values)

Try to print the resulting gradients

In [7]:
backpropagate(output, error)
update(output, learning_rate=learning_rate)

# we can print the computed gradients at each node.

print(output.errors[0])
print(weights_output.input)
print(biases_output.input)
print(weights_layer1.input)

[[0.06682728 0.05009681]]
[array([[ 0.79960459, -0.60029642],
       [-0.30034255,  0.69974321],
       [ 0.72250542,  0.3168711 ]])]
[array([[-0.54319681, -0.35741771]])]
[array([[ 0.10008362,  0.29985343, -0.89983224, -0.80006288],
       [ 0.40010452,  0.69981679,  0.3002097 ,  0.6999214 ],
       [-0.60014633, -0.3997435 , -0.60029358,  0.20011004]])]


Sample machine learning using a real dataset
============================================

Load the iris test data set. Objective is to classify Iris species using supplied features

In [8]:
train_set = []
train_set_labels = []
validation_set = []
validation_set_labels = []

label_encodings = {
  "Iris-setosa": [1., 0., 0.],
  "Iris-versicolor": [0., 1., 0.],
  "Iris-virginica": [0., 0., 1.],
}

with open(os.path.join('datasets', 'iris.data'),"r") as file:
  csv_rows = [x.strip().split(',') for x in file.readlines()]
  total_samples = len(csv_rows)
  test_set_indexes = random.sample(range(total_samples), k=int(total_samples * .3))
  for idx, sample in enumerate(csv_rows):
    row_values = [float(x) for x in sample[0:4]]
    row_label = sample[4]
    if idx in test_set_indexes:
      validation_set.append(row_values)
      validation_set_labels.append(label_encodings.get(row_label))
    else:
      train_set.append(row_values)
      train_set_labels.append(label_encodings.get(row_label))

Build Model with 2 hidden layers and one output layer

In [9]:
input = Node(np.array([[5.1,3.5,1.4,0.2]]))
learning_rate = 0.01

# set number of nodes per layer
NUM_HIDDEN_LAYER1 = 3
NUM_HIDDEN_LAYER2 = 2
NUM_OUTPUT_LAYER = 3

biases_layer1 = Node(np.random.uniform(size= [1,NUM_HIDDEN_LAYER1]), trainable=True)
weights_layer1 = Node(np.random.uniform(size=[4,NUM_HIDDEN_LAYER1]), trainable=True)
biases_layer2 = Node(np.random.uniform(size=[1,NUM_HIDDEN_LAYER2]), trainable=True)
weights_layer2 = Node(np.random.uniform(size=[NUM_HIDDEN_LAYER1,NUM_HIDDEN_LAYER2]), trainable=True)
biases_output = Node(np.random.uniform(size=[1,NUM_OUTPUT_LAYER]), trainable=True)
weights_output = Node(np.random.uniform(size=[NUM_HIDDEN_LAYER2,NUM_OUTPUT_LAYER]), trainable=True)


In [10]:
output_layer1 = Tanh(Add(MatMul(input, weights_layer1), biases_layer1))
output_layer2 = Tanh(Add(MatMul(output_layer1, weights_layer2), biases_layer2))
output = Sigmoid(Add(MatMul(output_layer2, weights_output), biases_output), logistic_slope=0.7)


Train the network on the training set and evaluate the mean square rror

In [11]:
epochs = 1000
best_error_rate = 1.0
for i  in range(epochs):
  for idx, sample in enumerate(train_set):
    input.update([sample])
    result = output.forward()
    error = [train_set_labels[idx]] - result
    backpropagate(output, error)
    update(output, learning_rate=learning_rate)
  
  #evaluate compute for sse
  error_rate = 0.
  for idx, test in enumerate(validation_set):
    input.update([test])
    result = output.forward()
    error_rate += np.sum(np.power([validation_set_labels[idx]] - result,2)) / 2
  error_rate /= len(validation_set)

  if (error_rate < best_error_rate):
    best_error_rate = error_rate
  if i%10==0:
    print(f"epoch: {i} best={best_error_rate} error={error_rate}")


epoch: 0 best=0.5648634885660312 error=0.5648634885660312
epoch: 10 best=0.3568732435085851 error=0.3568732435085851
epoch: 20 best=0.33599141554164486 error=0.33599141554164486
epoch: 30 best=0.3334450216626845 error=0.3334450216626845
epoch: 40 best=0.33307926838968854 error=0.33307926838968854
epoch: 50 best=0.3330076549065788 error=0.3330076549065788
epoch: 60 best=0.3329866561888947 error=0.3329866561888947
epoch: 70 best=0.33297848598406576 error=0.33297848598406576
epoch: 80 best=0.3329759593529792 error=0.3329759593529792
epoch: 90 best=0.33297593585513563 error=0.3329773649610218
epoch: 100 best=0.33297593585513563 error=0.33298219952363434
epoch: 110 best=0.33297593585513563 error=0.3329902845032541
epoch: 120 best=0.33297593585513563 error=0.3330015410768016
epoch: 130 best=0.33297593585513563 error=0.333015920010432
epoch: 140 best=0.33297593585513563 error=0.3330333723650572
epoch: 150 best=0.33297593585513563 error=0.33305383020054874
epoch: 160 best=0.33297593585513563 e